In [1]:
#import library
from pandas_gbq import read_gbq
import pandas as pd
import numpy as np
import os
import datetime
import ssl
import logging

In [ ]:
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta


# Get today's date
today = date.today()
idx = (today.weekday() + 1) % 7

# Current week (CY) range
end_date = today - timedelta(days=(7 + idx - 6))
start_date = end_date - timedelta(days=6)

# Prior period (PP) range
pp_end_date = start_date - timedelta(days=1)
pp_start_date = pp_end_date - timedelta(days=13)

# Corresponding week last year (CWLY and PWLY)
cwly_date = pp_end_date - timedelta(days=364) + timedelta(days=7)
pwly_date = pp_end_date - timedelta(days=364)

ytd_last_year = end_date - relativedelta(years=1)
ytd_current_year = date(end_date.year, 1, 1)

# Output all dates
(end_date, start_date, pp_end_date, pp_start_date, cwly_date, pwly_date,ytd_last_year,ytd_current_year)


In [ ]:
from custom_query import read_sql_query, sql_files

# Read SQL queries from files
queries = {key: read_sql_query(path) for key, path in sql_files.items()}

# Execute queries if all were successfully reada
if all(queries.values()):
    try:
        df_weekly_note = read_gbq(queries["weekly_note"], project_id='pcln-pl-airanalytics-prod')
        finance_data = read_gbq(queries["finance_data"], project_id='pcln-pl-airanalytics-prod')
        gds_incentives = read_gbq(queries["gds_incentives"], project_id='pcln-pl-airanalytics-prod')
        tsa_data = read_gbq(queries["tsa_data"], project_id='pcln-pl-airanalytics-prod')
        deal_share = read_gbq(queries["deal_share"], project_id='pcln-pl-airanalytics-prod')
        upsell_data = read_gbq(queries["upsell_query"], project_id='pcln-pl-airanalytics-prod')
        direct_parity_data = read_gbq(queries["direct_parity_data"], project_id='pcln-pl-airanalytics-prod')
        meta_parity_data = read_gbq(queries["meta_parity_data_query"], project_id='pcln-pl-airanalytics-prod')
        # bookability_data = read_gbq(queries["bookability_query"], project_id='pcln-pl-airanalytics-prod')
        roi_data = read_gbq(queries["roi_query"], project_id='pcln-pl-airanalytics-prod')
        dau_conversion_data = read_gbq(queries["dau_conversion_query"], project_id='pcln-pl-airanalytics-prod')
        midt_data = read_gbq(queries["midt_query"], project_id='pcln-pl-airanalytics-prod')
        sem_data = read_gbq(queries["sem_query"], project_id='pcln-pl-airanalytics-prod')
        print("SQL queries executed successfully.")
    except Exception as e:
        print(f"Failed to execute SQL queries: {e}")

## make a copy of dataset

In [4]:
# make a copy of every data set
df_weekly=df_weekly_note.copy()
df_finance=finance_data.copy()
df_gds_incentive=gds_incentives.copy()
df_tsa=tsa_data.copy()
df_direct_parity=direct_parity_data.copy()
df_meta_parity = meta_parity_data.copy()
df_roi=roi_data.copy()
dau_conversion=dau_conversion_data.copy()
df_deal_share=deal_share.copy()
df_upsell=upsell_data.copy()
# df_bookability=bookability_data.copy()
df_weekly.columns = df_weekly.columns.str.lower()
# df_midt=midt_data.copy()
# df_sem=sem_data.copy()


In [5]:
# df_gds_incentive=gds_incentives()

In [ ]:
df_gds_incentive

In [ ]:

base_dir = '../../data_file/'

# Check if directory exists
if not os.path.exists(base_dir):
    print(f"Error: Directory {base_dir} does not exist!")

# Save files
file_names = {
    'weekly': df_weekly,
    'df_finance': df_finance,
    'df_tsa': df_tsa,
    'direct_parity': df_direct_parity,
    'meta_parity': df_meta_parity,
    'dau_conversion': dau_conversion,
    'df_roi': df_roi,
    'deal_share': df_deal_share,
    # 'bookability': df_bookability,
 
}

# Save with verification
for name, df in file_names.items():
    file_path = f'{base_dir}{name}.csv'
    print(f"Saving: {file_path}")
    # print(df.head())  # Print first few rows to confirm data
    df.to_csv(file_path, index=False)

# Read files
for name in file_names.keys():
    file_path = f'{base_dir}{name}.csv'
    print(f"Reading: {file_path}")

    if os.path.exists(file_path):
        df_read = pd.read_csv(file_path)
        # print(df_read.head())  # Print first few rows to compare
    else:
        print(f"Error: {file_path} not found!")

In [ ]:
df_weekly

## Summary Table

In [9]:

def format_number(num):
    if pd.isna(num):
        return ''
    if abs(num) >= 1e6:
        return f"{num/1e6:.1f}M"
    elif abs(num) >= 1e3:
        return f"{num/1e3:.0f}K"
    elif abs(num) < 1e3:
        return "<1K"
    return f"{num:.0f}"

def format_percentage(num):
    if pd.isna(num):
        return ''
    return f"{num:.1f}%"

def format_percentage_2(num):
    if pd.isna(num):
        return ''
    return f"{num:.2f}%"

def round_to_nearest_10(num):
  return round(num / 10) * 10

df_pricelince=df_weekly[(df_weekly['brand']== 'Priceline')& (df_weekly['wk_ending']>= pp_end_date)& (df_weekly['wk_ending']<= end_date)]
df_pricelince_air=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['offer_type']== 'Flights Only')& (df_weekly['wk_ending']>= pp_end_date)& (df_weekly['wk_ending']<= end_date)]
df_pricelince_b2c=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')&(df_weekly['wk_ending']>= pp_end_date)& (df_weekly['wk_ending']<= end_date)]
df_pricelince_b2c_standalone=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')&(df_weekly['offer_type']== 'Flights Only')&(df_weekly['wk_ending']>= pp_end_date)& (df_weekly['wk_ending']<= end_date)]


In [10]:
import kpi
import importlib

importlib.reload(kpi)

from kpi import calculate_business_metrics

# Calculate business metrics
df_business = calculate_business_metrics(df_pricelince,end_date, pp_end_date)

from kpi import calculate_carrier_metrics
# Calculate carrier metrics
df_carrier = calculate_carrier_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_channel_metrics
# Calculate channel metrics
df_channel = calculate_channel_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_source_metrics
# Calculate source metrics
df_source = calculate_source_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_brand_metrics
# Calculate brand metrics
df_brand = calculate_brand_metrics(df_weekly, end_date, pp_end_date)


In [ ]:
df_business

In [ ]:
df_channel


In [ ]:
df_carrier

In [ ]:
df_source

# MOR


In [ ]:
# Create df_mor
df_mor= pd.DataFrame()
df_mor['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)


# Calculate YoY and YoY PW
df_mor.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor['Net Tickets'].sum(),df_mor['Net Tickets_cwly'].sum(),df_mor['Net Tickets_PW'].sum(), df_mor['Net Tickets_PWly'].sum()]


# Create df_mor_1
df_mor_1= pd.DataFrame()
df_mor_1['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
# Calculate YoY and YoY PW
df_mor_1.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_1['Net Tickets'].sum(),df_mor_1['Net Tickets_cwly'].sum(),df_mor_1['Net Tickets_PW'].sum(), df_mor_1['Net Tickets_PWly'].sum()]

df_mor = pd.concat([df_mor,df_mor_1.add_suffix('_total')], axis=1)


df_mor['Actual']=(df_mor['Net Tickets']/df_mor['Net Tickets_total']*100).round(1).apply(format_percentage)
df_mor['%mor_ly']=(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100).round().apply(format_percentage)
df_mor['YoY_bps']=(((df_mor['Net Tickets']/df_mor['Net Tickets_total']*100)-(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100))*100).round()
df_mor['Actual_PW']=(df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100).round().apply(format_percentage)
df_mor['YoY_PW_bps']=(((df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100)-(df_mor['Net Tickets_PWly']/df_mor['Net Tickets_PWly_total']*100))*100).round()
df_mor['YoY_bps']=round_to_nearest_10(df_mor['YoY_bps'])
df_mor['YoY_PW_bps']=round_to_nearest_10(df_mor['YoY_PW_bps'])
df_mor=df_mor.loc[['Retail (Disclosed)', 'Total'],['Actual','YoY_bps','YoY_PW_bps']]
df_mor


In [ ]:
# Create df_mor
df_mor_2= pd.DataFrame()
df_mor_2['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_2['Net Tickets'].sum(),df_mor_2['Net Tickets_cwly'].sum(),df_mor_2['Net Tickets_PW'].sum(), df_mor_2['Net Tickets_PWly'].sum()]
actual_cy=(df_mor_2.loc['US Outbound','Net Tickets']*100/df_mor_2.loc['Total','Net Tickets']).round()
actual_ly=df_mor_2.loc['US Outbound','Net Tickets_cwly']*100/df_mor_2.loc['Total','Net Tickets_cwly']
actual_PW=df_mor_2.loc['US Outbound','Net Tickets_PW']*100/df_mor_2.loc['Total','Net Tickets_PW']
actual_PWly=df_mor_2.loc['US Outbound','Net Tickets_PWly']*100/df_mor_2.loc['Total','Net Tickets_PWly']
df_mor.loc['US Outbound','Actual']=format_percentage(actual_cy)
df_mor.loc['US Outbound','YoY_bps']=round_to_nearest_10((actual_cy-actual_ly)*100)
df_mor.loc['US Outbound','YoY_PW_bps']=round_to_nearest_10((actual_PW-actual_PWly)*100)
new_index_order = ['US Outbound','Retail (Disclosed)','Total']
df_mor=df_mor.reindex(new_index_order).fillna('')
df_mor

##  DAU / ROI


In [ ]:

from dau_roi_table import calculate_dau_conversion
from dau_roi_table import calculate_roi
from dau_roi_table import create_roi_table

import importlib, dau_roi_table
importlib.reload(dau_roi_table)



# 1) Compute raw DAU/Conversion
df_dau_converison= calculate_dau_conversion(dau_conversion,format_percentage,end_date, pp_end_date, cwly_date, pwly_date)
df_dau_converison

# 2) Compute ROI
df_roi_section = calculate_roi(df_roi, end_date, pp_end_date, cwly_date, pwly_date)
df_roi_section


# # 3) Build the presentation table
df_roi_v = create_roi_table(df_roi_section,df_dau_converison,end_date, pp_end_date, cwly_date, pwly_date,format_number)
df_roi_v['Conversion'] =(df_roi_v['Conversion']*100).apply(format_percentage)

df_roi_v


## Parity data

In [ ]:
from parity_table import create_parity_table

df_parity = create_parity_table(
        format_percentage,
        df_direct_parity,
        round_to_nearest_10,
        df_meta_parity)
df_parity

## Deal Share

In [ ]:
deal_ticket_actual_cy=df_deal_share[(df_deal_share['wk_ending']==end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cy=deal_ticket_actual_cy[0]/(deal_ticket_actual_cy[0]+deal_ticket_actual_cy[1])
# deal_ticket_actual_cy=format_percentage(deal_ticket_actual_cy*100)

deal_ticket_actual_cy

deal_ticket_actual_pw=df_deal_share[(df_deal_share['wk_ending']==pp_end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pw=deal_ticket_actual_pw[0]/(deal_ticket_actual_pw[0]+deal_ticket_actual_pw[1])
# deal_ticket_actual_pw=format_percentage(deal_ticket_actual_pw*100)
deal_ticket_actual_pw

deal_ticket_actual_cwly=df_deal_share[(df_deal_share['wk_ending']==cwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cwly=deal_ticket_actual_cwly[0]/(deal_ticket_actual_cwly[0]+deal_ticket_actual_cwly[1])
# deal_ticket_actual_cwly=format_percentage(deal_ticket_actual_cwly*100)
deal_ticket_actual_cwly

deal_ticket_actual_pwly=df_deal_share[(df_deal_share['wk_ending']==pwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pwly=deal_ticket_actual_pwly[0]/(deal_ticket_actual_pwly[0]+deal_ticket_actual_pwly[1])
# deal_ticket_actual_pwly=format_percentage(deal_ticket_actual_pwly*100)
deal_ticket_actual_cy,deal_ticket_actual_cwly,deal_ticket_actual_pw,deal_ticket_actual_pwly

## MIDT

In [20]:



# df_midt_data= pd.DataFrame()
# #Calulate actual
# a=df_midt[(df_midt['wk_ending']==end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==end_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual = ((a * 100) / b)


# #Calulate cwly_date
# a=df_midt[(df_midt['wk_ending']==cwly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==cwly_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual_cwly = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==pp_end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==pp_end_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PW = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==PWly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==PWly_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PWly = ((a * 100) / b)


# # Create a new DataFrame with the calculated 'Actual' column
# df_midt_data = pd.DataFrame()
# df_midt_data['Actual']=[format_percentage(Actual)]
# df_midt_data['YoY (bps)']=[round_to_nearest_10((Actual-Actual_cwly)*100)]
# df_midt_data['YoY PW(bps)']=[round_to_nearest_10((actual_PW-actual_PWly)*100)]
# df_midt_data
# # df_midt_data['Actual']

## SEM

In [21]:
# df_sem=df_sem[(df_sem['DOMAIN']=='You')|(df_sem['DOMAIN']=='expedia.com')]
# df_sem
# # Calculate actuals for current, cwly, pp, and pwly dates
# def compute_rate_ratio(df, date):
#     grouped = df[df['wk_ending'] == date].groupby('DOMAIN')[['ABS_TOP_PAGE_RATE_IMPRESSIONS', 'TOTAL_IMPRESSIONS']].sum()
#     grouped['RATE_RATIO'] = grouped['ABS_TOP_PAGE_RATE_IMPRESSIONS'] / grouped['TOTAL_IMPRESSIONS']
#     return grouped['RATE_RATIO']  # return only the RATE_RATIO series

# # Compute each set
# actual = compute_rate_ratio(df_sem, end_date)
# actual_cwly = compute_rate_ratio(df_sem, cwly_date)
# actual_pw = compute_rate_ratio(df_sem, pp_end_date)
# actual_pwly = compute_rate_ratio(df_sem, pwly_date)

# df_sem_data = pd.DataFrame({
#     'Actual': (actual*100).apply(format_percentage),
#     'YoY (bps)': round_to_nearest_10((actual - actual_cwly) * 10000),
#     'YoY PW(bps)': round_to_nearest_10((actual_pw - actual_pwly) * 10000)
# })

# df_sem_data.reset_index(inplace=True)  # optional, for clarity if you need DOMAIN as a column
# df_sem_data


In [ ]:
df_weekly.loc[
       (df_weekly['company'].str.contains('priceline', case=False, na=False))
       & (df_weekly['offer_type'].str.contains('Flights', case=False, na=False))]

In [ ]:
total_priceline_ytd

In [ ]:
df_gds_incentive['net_gds_incentives']=df_gds_incentive['net_gds_incentives'].astype(int)

total_priceline_ytd = (
    df_weekly.loc[
        (df_weekly['wk_ending'] <= end_date)
        & (df_weekly['wk_ending'] >= ytd_current_year)
        & (df_weekly['company'].str.contains('priceline', case=False, na=False))]
        [['net_tkts_cy', 'net_tkts_ly','gr_tkts_cy', 'gr_tkts_ly', 'net_contribution_cy', 'net_contribution_ly',
        'gross_contribution_cy', 'gross_contribution_ly','normalized_net_tickets_cy','normalized_net_tickets_ly',
        'normalized_gross_tickets_cy','normalized_gross_tickets_ly','net_contr_fee_cy','net_contr_fee_ly','gr_contr_fee_cy','gr_contr_fee_ly']]
    .sum()
    .to_frame()
    .T
)
total_priceline_ytd_flightonly = (
    df_weekly.loc[
        (df_weekly['brand'].str.contains('Priceline', case=False, na=False))
        & (df_weekly['offer_type'].str.contains('Flights', case=False, na=False))]
        [['net_tkts_cy','net_tkts_ly','net_contr_fee_cy','net_contr_fee_ly','gr_contr_fee_cy','gr_contr_fee_ly']]
    .sum()
    .to_frame()
    .T
)

total_priceline_ytd['company'] = 'Priceline YTD Total'

total_priceline_ytd = total_priceline_ytd[
    ['company', 'net_tkts_cy', 'net_tkts_ly', 'gr_tkts_cy', 'gr_tkts_ly','net_contribution_cy'
    , 'net_contribution_ly', 'gross_contribution_cy', 'gross_contribution_ly',
    'normalized_net_tickets_cy','normalized_net_tickets_ly','normalized_gross_tickets_cy'
    ,'normalized_gross_tickets_ly', 'net_contr_fee_cy','net_contr_fee_ly','gr_contr_fee_cy','gr_contr_fee_ly']
]
total_priceline_ytd['ytd_net'] = (
    total_priceline_ytd['net_tkts_cy'].fillna(0) / total_priceline_ytd['net_tkts_ly'].replace(0, pd.NA)
) - 1
total_priceline_ytd['ytd_gr'] = (
    total_priceline_ytd['gr_tkts_cy'].fillna(0) / total_priceline_ytd['gr_tkts_ly'].replace(0, pd.NA)
) - 1

total_priceline_ytd['ytd_netrev'] = (
    (total_priceline_ytd['net_contribution_cy'].fillna(0)) / (total_priceline_ytd['net_contribution_ly'].replace(0, pd.NA))
) - 1
total_priceline_ytd['ytd_grrev'] = (
    (total_priceline_ytd['gross_contribution_cy']) / (total_priceline_ytd['gross_contribution_ly'].replace(0, pd.NA))
) - 1

total_priceline_ytd['ytd_nornet'] = (
    total_priceline_ytd['normalized_net_tickets_cy'].fillna(0) / total_priceline_ytd['normalized_net_tickets_ly'].replace(0, pd.NA)
) - 1

total_priceline_ytd['ytd_norgr'] = (
    total_priceline_ytd['normalized_gross_tickets_cy'].fillna(0) / total_priceline_ytd['normalized_gross_tickets_ly'].replace(0, pd.NA)
) - 1

total_priceline_ytd['ytd_netconrfee'] = (
    (total_priceline_ytd['net_contr_fee_cy'].fillna(0)+df_gds_incentive['net_gds_incentives'][0]) / (total_priceline_ytd['net_contr_fee_ly'].replace(0, pd.NA)+df_gds_incentive['net_gds_incentives'][1])
) - 1

total_priceline_ytd['ytd_grconrfee'] = (
    (total_priceline_ytd['gr_contr_fee_cy'].fillna(0)+df_gds_incentive['net_gds_incentives'][0]) / (total_priceline_ytd['gr_contr_fee_ly'].replace(0, pd.NA)+df_gds_incentive['net_gds_incentives'][1])
) - 1

total_priceline_ytd['ytd_netconrfee_flightonly'] = (
    (total_priceline_ytd_flightonly['net_contr_fee_cy'].fillna(0)+df_gds_incentive['net_gds_incentives'][0]) / (total_priceline_ytd_flightonly['net_contr_fee_ly'].replace(0, pd.NA)+df_gds_incentive['net_gds_incentives'][1])
) - 1

total_priceline_ytd['ytd_grconrfee_flightonly'] = (
    (total_priceline_ytd_flightonly['gr_contr_fee_cy'].fillna(0)+df_gds_incentive['net_gds_incentives'][0]) / (total_priceline_ytd_flightonly['gr_contr_fee_ly'].replace(0, pd.NA)+df_gds_incentive['net_gds_incentives'][1])
) - 1

summary_data = [
{
    'Measure': 'Net Tickets',
    'CY': total_priceline_ytd['net_tkts_cy'].iloc[0],
    'LY': total_priceline_ytd['net_tkts_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_net'].iloc[0]*100
},
{
    'Measure': 'Gross Tickets',
    'CY': total_priceline_ytd['gr_tkts_cy'].iloc[0],
    'LY': total_priceline_ytd['gr_tkts_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_gr'].iloc[0]*100
},
{
    'Measure': 'Net Revenue(net_contribution_cy)',
    'CY': total_priceline_ytd['net_contribution_cy'].iloc[0],
    'LY': total_priceline_ytd['net_contribution_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_netrev'].iloc[0]*100
},
{
    'Measure': 'Gross Revenue(gross_contribution_cy)',
    'CY': total_priceline_ytd['gross_contribution_cy'].iloc[0],
    'LY': total_priceline_ytd['gross_contribution_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_grrev'].iloc[0]*100
},
{
    'Measure': 'Normalized Net Tickets',
    'CY': total_priceline_ytd['normalized_net_tickets_cy'].iloc[0],
    'LY': total_priceline_ytd['normalized_net_tickets_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_nornet'].iloc[0]*100
},
{
    'Measure': 'Normalized Gross Tickets',
    'CY': total_priceline_ytd['normalized_gross_tickets_cy'].iloc[0],
    'LY': total_priceline_ytd['normalized_gross_tickets_ly'].iloc[0],
    'YTD': total_priceline_ytd['ytd_norgr'].iloc[0]*100
},
{
    'Measure': 'Net Contribution+ Fee',
    'CY': total_priceline_ytd['net_contr_fee_cy'].iloc[0]
    # +df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd['net_contr_fee_ly'].iloc[0]
    # +df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_netconrfee'].iloc[0]*100
},
{
    'Measure': 'Gross Contribution + Fee',
    'CY': total_priceline_ytd['gr_contr_fee_cy'].iloc[0]
    # +df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd['gr_contr_fee_ly'].iloc[0]
    # +df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_grconrfee'].iloc[0]*100 
},
{
    'Measure': 'GDS Incentive',
    'CY': df_gds_incentive['net_gds_incentives'][0],
    'LY': df_gds_incentive['net_gds_incentives'][1],
    # 'YTD': total_priceline_ytd['ytd_gds_incentive'].iloc[0]*100
},
{
    'Measure': 'Net Cont + Fee + Incentives',
    'CY': total_priceline_ytd['net_contr_fee_cy'].iloc[0]+df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd['net_contr_fee_ly'].iloc[0]+df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_netconrfee'].iloc[0]*100
},
{
    'Measure': 'Gross Cont + Fee + Incentives',
    'CY': total_priceline_ytd['gr_contr_fee_cy'].iloc[0]+df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd['gr_contr_fee_ly'].iloc[0]+df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_grconrfee'].iloc[0]*100    
},
{
    'Measure': 'Net Cont + Fee + Incentives(Flight Only)',
    'CY': total_priceline_ytd_flightonly['net_contr_fee_cy'].iloc[0]+df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd_flightonly['net_contr_fee_ly'].iloc[0]+df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_netconrfee_flightonly'].iloc[0]*100
},
{
    'Measure': 'Gross Cont + Fee + Incentives(Flight Only)',
    'CY': total_priceline_ytd_flightonly['gr_contr_fee_cy'].iloc[0]+df_gds_incentive['net_gds_incentives'][0]
    ,
    'LY': total_priceline_ytd_flightonly['gr_contr_fee_ly'].iloc[0]+df_gds_incentive['net_gds_incentives'][1]
    ,
    'YTD': total_priceline_ytd['ytd_grconrfee_flightonly'].iloc[0]*100    
}
]

# Create DataFrame
summary_table = pd.DataFrame(summary_data)

# Format numeric columns
summary_table['CY'] = summary_table['CY'].round(0)
summary_table['LY'] = summary_table['LY'].round(0)
summary_table['YTD'] = summary_table['YTD']
summary_table


# Summary Table

## YTD

In [ ]:
from summary_table_apl import create_ytd_ly
from summary_table_apl import create_subsummary_table
# importlib.reload(summary_table_apl)

create_ytd_ly=create_ytd_ly(df_weekly,df_gds_incentive,end_date,ytd_current_year)

create_ytd_ly=create_ytd_ly.set_index('Measure')
create_ytd_ly

In [ ]:
rows = [
    'Net Tickets','Gross Tickets',
    'Net Cont + Fee + Incentives(Flight Only)','Gross Cont + Fee + Incentives(Flight Only)',
    'Normalized Net Tickets','Normalized Gross Tickets',
]

name_to_code = {
    'Net Tickets':'net_tkts_cy',
    'Gross Tickets':'gr_tkts_cy',
    'Net Cont + Fee + Incentives(Flight Only)':'net_contribution_cy',
    'Gross Cont + Fee + Incentives(Flight Only)':'gross_contribution_cy',
    'Normalized Net Tickets':'normalized_net_tickets_cy',
    'Normalized Gross Tickets':'normalized_gross_tickets_cy',
}

subset = (
    create_ytd_ly
      .loc[rows, 'YTD']          # select rows + single column
      .rename(index=name_to_code) # rename to codes
      .rename('YTD')  # rename index to codes
)

subset


In [ ]:
import summary_table_apl as summary_table_apl

from summary_table_apl import create_subsummary_table
importlib.reload(summary_table_apl)

df_subsummary = create_subsummary_table(df_pricelince,create_ytd_ly,df_pricelince_air,format_percentage,format_number)

# df_subsummary=df_subsummary.reset_index().rename(columns={'index': 'Measure'}).set_index('Measure') 
  
df_subsummary['YTD']=subset.apply(format_percentage)
df_subsummary


In [28]:
df_normalized=df_subsummary.tail(2)
df_normalized = (
    df_normalized
    .reset_index()                 
    .rename(columns={'index': 'Measure'})  
    .set_index('Measure')          
)


df_normalized=df_normalized

In [ ]:
# importlib.reload(summary_table_finance)
# importlib.reload(summary_table_finance)
from summary_table_finance import create_ly_table
from summary_table_finance import create_plan_table

df_summary_ly = create_ly_table(df_finance,df_subsummary,format_percentage,format_number)


df_summary_ly=df_summary_ly.set_index('Measure')
df_normalized.index.name = 'Measure'
df_summary_ly['YTD']= df_subsummary['YTD'][:-2]
df_summary_ly



df_summary_plan = create_plan_table(df_finance,df_summary_ly,format_percentage,format_number)
df_summary_plan

In [ ]:

df_finance_result= pd.merge(df_summary_ly, df_summary_plan,how='left', on='Measure',suffixes=('','_pl'))
df_finance_result = df_finance_result.rename(columns={
        'Reporting Week_ly': 'Reporting Week',
        'Previous Week_ly': 'Previous Week'
        })
df_finance_result=df_finance_result.fillna('').set_index('Measure')
df_finance_result


In [ ]:
df_normalized

In [ ]:
df_summary=pd.concat([df_finance_result, df_normalized]).fillna('')
df_summary

## TSA

In [ ]:
tsa_cy=df_tsa[df_tsa['wk_ending']==end_date]['tsa_passengers'].sum()
pcln_cy=df_tsa[df_tsa['wk_ending']==end_date]['pcln_passengers'].sum()
tsa_actual_cy=round((pcln_cy*100/tsa_cy),2)


tsa_pw=df_tsa[df_tsa['wk_ending']==pp_end_date]['tsa_passengers'].sum()
pcln_pw=df_tsa[df_tsa['wk_ending']==pp_end_date]['pcln_passengers'].sum()
tsa_actual_pw=round((pcln_pw*100/tsa_pw),2)


tsa_cwly=df_tsa[df_tsa['wk_ending']==cwly_date]['tsa_passengers'].sum()
pcln_cwly=df_tsa[df_tsa['wk_ending']==cwly_date]['pcln_passengers'].sum()
tsa_actual_cwly=round((pcln_cwly*100/tsa_cwly),2)



tsa_pwly=df_tsa[df_tsa['wk_ending']==pwly_date]['tsa_passengers'].sum()
pcln_pwly=df_tsa[df_tsa['wk_ending']==pwly_date]['pcln_passengers'].sum()
tsa_actual_pwly=round((pcln_pwly*100/tsa_pwly),2)


tsa_ytd=df_tsa[(df_tsa['date'] <=end_date)&(df_tsa['date']>=pd.to_datetime('2025-01-01'))]['tsa_passengers'].sum()
pcln_ytd=df_tsa[(df_tsa['date']<=end_date)& (df_tsa['date']>=pd.to_datetime('2025-01-01'))]['pcln_passengers'].sum()
tsa_actual_ytd=round((pcln_ytd*100/tsa_ytd),2)


tsa_ytd_ly=df_tsa[(df_tsa['date'] <=ytd_last_year)]['tsa_passengers'].sum()
pcln_ytd_ly=df_tsa[(df_tsa['date']<=ytd_last_year)]['pcln_passengers'].sum()
tsa_actual_ytd_ly=round((pcln_ytd_ly*100/tsa_ytd_ly),2)

print("TSA CY",tsa_cy,"PCLN CY",pcln_cy,"%TSA",tsa_actual_cy)
print("TSA LY",tsa_cwly,"PCLN LY",pcln_cwly,"%TSA",tsa_actual_cwly)
print("TSA PW",tsa_pw,"PCLN PW",pcln_pw,"%TSA",tsa_actual_pw)
print("TSA PWLY",tsa_pwly,"PCLN PWly",pcln_pwly,"%TSA",tsa_actual_pwly)
print("TSA YTD",tsa_ytd,"PCLN CY",pcln_ytd,"%TSA YTD",tsa_actual_ytd)
print("TSA YTD LY",tsa_ytd_ly,"PCLN YTD LY",pcln_cwly,"%TSA",tsa_actual_ytd_ly)

In [ ]:
# df_summary=df_summary.reset_index(names=['Metric'])

df_summary.loc['DAU','Actual']=df_roi_v.iloc[:, 0].loc['Total']
df_summary.loc['DAU','Reporting Week']=df_roi_v.iloc[:, 1].loc['Total']
df_summary.loc['DAU','Previous Week']=df_roi_v.iloc[:, 2].loc['Total']



df_summary.loc['TSA','Actual']=format_percentage_2(tsa_actual_cy)
df_summary.loc['TSA','Reporting Week']=format_percentage((tsa_actual_cy/tsa_actual_cwly-1)*100)
df_summary.loc['TSA','Previous Week']=format_percentage((tsa_actual_pw/tsa_actual_pwly-1)*100)
df_summary.loc['TSA','YTD']=format_percentage((tsa_actual_ytd/tsa_actual_ytd_ly-1)*100)

df_summary=df_summary.fillna('')


df_summary

In [219]:
# from tsa_table import create_tsa_web_table
# df_tsa_web=create_tsa_web_table()
# df_tsa_web=df_tsa_web.dropna()

# import tsa_table
# import importlib

# importlib.reload(tsa_table)
# from tsa_table import create_df_tsa_table

# df_tsa=create_df_tsa_table(tsa_data,df_tsa_web,format_percentage,end_date,pp_end_date,cwly_date,pwly_date)
# df_tsa


In [ ]:
df_upsell

In [ ]:
df_upsell[(df_upsell['week_ending']==end_date)]

## Upsell

In [ ]:
# df_upsell
upsell_cy=df_upsell[(df_upsell['week_ending']==end_date)]['upsell_tickets'].sum()
select_cy=df_upsell[(df_upsell['week_ending']==end_date)]['Selected_tickets'].sum()
upsell_rate_cy=upsell_cy/(upsell_cy+select_cy)


upsell_pw=df_upsell[(df_upsell['week_ending']==pp_end_date)]['upsell_tickets'].sum()
select_pw=df_upsell[(df_upsell['week_ending']==pp_end_date)]['Selected_tickets'].sum()
upsell_rate_pw=upsell_pw/(upsell_pw+select_pw)



upsell_cwly=df_upsell[(df_upsell['week_ending']==cwly_date)]['upsell_tickets'].sum()
select_cwly=df_upsell[(df_upsell['week_ending']==cwly_date)]['Selected_tickets'].sum()
upsell_rate_cwly=upsell_cwly/(upsell_cwly+select_cwly)


upsell_pwly=df_upsell[(df_upsell['week_ending']==pwly_date)]['upsell_tickets'].sum()
select_pwly=df_upsell[(df_upsell['week_ending']==pwly_date)]['Selected_tickets'].sum()
upsell_rate_pwly=upsell_pwly/(upsell_pwly+select_pwly)

upsell_rate_cy,upsell_rate_pw,upsell_rate_cwly,upsell_rate_pwly

In [ ]:
df_roi_v

In [ ]:
df_others = pd.DataFrame()

# Deal Share
df_others.loc['Deal Share','Actual_pcln'] = format_percentage(deal_ticket_actual_cy * 100)
df_others.loc['Deal Share','YoY (bps)_pcln'] = round_to_nearest_10((deal_ticket_actual_cy - deal_ticket_actual_cwly) * 10000)
df_others.loc['Deal Share','YoY PW (bps)_pcln'] = round_to_nearest_10((deal_ticket_actual_pw - deal_ticket_actual_pwly) * 10000)

# Upsell
df_others.loc['Upsell','Actual_pcln'] = format_percentage(upsell_rate_cy*100)
df_others.loc['Upsell','YoY (bps)_pcln'] = round_to_nearest_10((upsell_rate_cy - upsell_rate_cwly)*10000)
df_others.loc['Upsell','YoY PW (bps)_pcln'] = round_to_nearest_10((upsell_rate_pw - upsell_rate_pwly)*10000)

# Conversion
df_others.loc['Conversion','Actual_pcln'] = df_roi_v.iloc[-1, 3]

df_others.loc['Conversion','YoY (bps)_pcln'] = round_to_nearest_10((df_dau_converison['conversion']-df_dau_converison['conversion_cwly'])* 10000)['Total']
df_others.loc['Conversion','YoY PW (bps)_pcln'] = round_to_nearest_10((df_dau_converison['conversion_pw']-df_dau_converison['conversion_pwly'])* 10000)['Total']


# df_others.loc['MIDT','Actual_pcln']=df_midt_data.loc[0,'Actual']
# df_others.loc['MIDT','YoY (bps)_pcln']=df_midt_data.loc[0,'YoY (bps)']
# df_others.loc['MIDT','YoY PW (bps)_pcln']=df_midt_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_pcln']=df_sem_data.loc[0,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_pcln']=df_sem_data.loc[0,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_pcln']=df_sem_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_expe']=df_sem_data.loc[1,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_expe']=df_sem_data.loc[1,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_expe']=df_sem_data.loc[1,'YoY PW(bps)']

# US Outbound & Merchant
df_others.loc['US Outbound','Actual_pcln'] = df_mor.loc['US Outbound','Actual']
df_others.loc['US Outbound','YoY (bps)_pcln'] = df_mor.loc['US Outbound','YoY_bps']
df_others.loc['US Outbound','YoY PW (bps)_pcln'] = df_mor.loc['US Outbound','YoY_PW_bps']

df_others.loc['Merchant Retail','Actual_pcln'] = df_mor.loc['Retail (Disclosed)','Actual']
df_others.loc['Merchant Retail','YoY (bps)_pcln'] = df_mor.loc['Retail (Disclosed)','YoY_bps']
df_others.loc['Merchant Retail','YoY PW (bps)_pcln'] = df_mor.loc['Retail (Disclosed)','YoY_PW_bps']

df_others.loc['Merchant Total','Actual_pcln'] = df_mor.loc['Total','Actual']
df_others.loc['Merchant Total','YoY (bps)_pcln'] = df_mor.loc['Total','YoY_bps']
df_others.loc['Merchant Total','YoY PW (bps)_pcln'] = df_mor.loc['Total','YoY_PW_bps']

# Parity
df_others = pd.concat([df_others, df_parity.iloc[0:]])

# Reorder index
new_index_order = [
    'ARC-PCLN','ARC-Agoda','ARC-B.com','ARC-BHI'
    ,'MIDT'
    ,'SEM Impressions',
    'Direct vs Expedia','Kayak Placement','Skyscanner Placement',
    'US Outbound','Merchant Retail','Merchant Total',
    'Deal Share','Conversion','Bookability','Upsell','Booking rate','Tickets/order'
]
df_others = df_others.reindex(new_index_order).fillna('')
df_others

In [ ]:

import config_table
import importlib

importlib.reload(config_table)
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Pt
from config_table import clear_document
from config_table import set_font
from config_table import create_word_table
from config_table import create_summary_table
from config_table import create_others_table
# from config_table import create_dau_table
from config_table import create_roi_table


# Data preparation and configurations
carrier_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Retail.jpg', 'title': 'Retail'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Express.png', 'title': 'Express Deals'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

business_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Standalone.png', 'title': 'Standalone'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Package.png', 'title': 'Package'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

channel_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/App.png', 'title': 'App'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/MWeb Desktop.png', 'title': 'Web'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

source_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Published.png', 'title': 'Published'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Private.png', 'title': 'Private'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

def customize_bullet_style(paragraph, font_size):
    """
    Customize bullet style in a paragraph using XML manipulation.
    """
    # Access the paragraph's properties
    pPr = paragraph._element.get_or_add_pPr()
    numPr = OxmlElement('w:numPr')
    ilvl = OxmlElement('w:ilvl')
    ilvl.set(qn('w:val'), '0')  # Set indentation level
    numId = OxmlElement('w:numId')
    numId.set(qn('w:val'), '1')  # Set numbering ID

    numPr.append(ilvl)
    numPr.append(numId)
    pPr.append(numPr)

    # Modify font properties
    for run in paragraph.runs:
        run.font.name = "Montserrat"
        run.font.size = font_size

# Create a Word Document
word_document = Document()

# Clear the document (if necessary)
clear_document(word_document)

# Add Title
word_document.add_paragraph()
word_document.add_paragraph(f'Summary')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
word_document.add_paragraph('\n')

# Create Summary Table
create_summary_table(word_document,df_summary)

# Add General Notes
summary_notes = [
    'All Priceline tickets (includes B2B, Package, Express Deals, Phone Sales)',
    'Normalized tickets are counted the same as tickets, except split tickets count as 1 instead of 2',
    'Refunds are assigned to refund date',
    'Revenue is contribution w/fee + GDS incentives.  Does not include package or phone sales.',
    'Daily Active Users: engaged customers in GA4',
    'TSA market share: travel date; numerator counts all slices (OW is 1 slice; RT is 2 slices) where the first segment is either US domestic or US outbound (Priceline-only); denominator includes all people passing through TSA screening machines'
]
for note in summary_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
    customize_bullet_style(word_document.paragraphs[-1], font_size=Pt(7))

# Add Tables for Business, Carrier, Channel, and Source
tables = [
    (df_business, "Business", business_logos, [
        'All Priceline tickets (includes Express Deals and Phone Sales, not normalized)'
    ]),
    (df_carrier, "Carrier", carrier_logos, [
        'Priceline (including phone sales), B2C, Standalone (not normalized)'
    ]),
    (df_channel, "Channel", channel_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)'
    ]),
    (df_source, "Source", source_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)',
        'Indirect Connect = GDS + Aggregators + Consolidators + NDC-X'
    ])
]
for df, title, logos, notes in tables:
    if df is None:
        continue

   # Add specific custom notes for each table
    if title == "Business":
        custom_note = "Total Business - Detail"
    elif title == "Carrier":
        custom_note = "Priceline B2C Standalone - Detail"
    else:
        custom_note = " "

    # Add the custom note before the table
    word_document.add_paragraph(custom_note)
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
    word_document.add_paragraph()

    create_word_table(df, title, logos, word_document)

    # word_document.add_paragraph()

    for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        # word_document.add_paragraph('\n')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)

word_document.add_paragraph('\n')
word_document.add_paragraph('Other Metrics')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
create_others_table(word_document, df_others)

others_notes = [
    'ARC market share: OTA only, US POS, does not include NK/F9/SY',
    'SEM impression share: percentage of eligible impressions we show at the absolute top of the ads',
    'Direct parity vs Expedia: US origins, win+tie rate',
    'Kayak placement: US origins, availability in the top 4 positions',
    'Skyscanner placement: US origins, availability in the top 4 positions',
    'US outbound share: ticket share for US origins, international destinations',
    'Merchant share: ticket share where Priceline is merchant of record',
    'Deal share: share of tickets sold as a deal',
    'Conversion: GA4, converted customers / engaged customers',
    'Bookability: acceptance rate, includes multiple attempts by same user',
    'Upsell: share of tickets upsold when there was an upsell opportunity',
    'Booking rate: net tickets/gross tickets',
    'Tickets/order: net tickets/net orders'


]
for note in others_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')

word_document.add_paragraph(f'ROI')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)

# Add ROI Section
# roi_notes = [
#     'ROI = Contribution / Cost',
#     'SEM Brand includes all products, weighted by ratio of flight orders',
#     'Meta includes Kayak/Momondo, includes Kayak credit',
#     'Last week'
# ]
create_roi_table(word_document, df_roi_v.iloc[:-1])

for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')


# Save the Document
output_filename = os.path.join('../output/', f'Flight Performance Week Ending {end_date}.docx')
word_document.save(output_filename)
print(f"Word document '{output_filename}' has been created successfully.")

# Save PDF to Shared Drive
share_drive_path = '../../../Flight Weekly Note Output/'
word_document.save(share_drive_path + f'Flight Performance Week Ending {end_date}.pdf')

print(f"Word document saved to shared drive at '{share_drive_path}' has been created successfully.")


In [ ]:
df_summary